In [16]:
!pip -q install langchain langchain_community
!pip -q install bitsandbytes accelerate
!pip -q install datasets loralib
!pip -q install pypdf

In [17]:
!pip -q install unstructured
!pip install tokenizers
!pip install xformers
!pip install pinecone

In [11]:
!pip uninstall -y transformers
!pip install transformers==4.37.2

Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [28]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone as LangChainPinecone # Import Pinecone from langchain.vectorstores with an alias
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [11]:
import nltk

In [12]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [13]:
URLs=[
    'https://www.artificialintelligence-news.com/news/openai-rejects-robinhood-unauthorised-tokenised-shares/',
    'https://www.artificialintelligence-news.com/news/study-finds-ai-slash-global-carbon-emissions/',
    'https://www.artificialintelligence-news.com/news/nvidia-reclaims-title-of-most-valuable-company-on-ai-momentum/',
    'https://www.artificialintelligence-news.com/news/meta-will-train-ai-models-using-eu-user-data/'
]

In [14]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [15]:
len(data)

4

In [16]:
text_spiltter = CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=200)

In [17]:
text_chunks = text_spiltter.split_documents(data)

In [1]:
!pip install peft==0.10.0


  Using cached peft-0.10.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Attempting uninstall: peft
    Found existing installation: peft 0.16.0
    Uninstalling peft-0.16.0:
      Successfully uninstalled peft-0.16.0


In [2]:
!pip show peft


Name: peft
Version: 0.10.0
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: sourab@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: accelerate, huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 


In [5]:
embeddings = HuggingFaceEmbeddings()

/tmp/ipython-input-5-3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-5-3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [43]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
query_result = embeddings.embed_query("Hello World")
len(query_result)

768

In [49]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY','pcsk_21CN6p_FbzLvzi7aRq9jQzymqBwmGEiumQPb1pq3TdxkEsNuHTfJfEiRvm8tHbdtq3kgLw')


In [50]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = 'website-bot'

In [53]:
!pip install -qU langchain-pinecone

In [62]:
text_chunks

[Document(metadata={'source': 'https://www.artificialintelligence-news.com/news/openai-rejects-robinhood-unauthorised-tokenised-shares/'}, page_content='403 - Forbidden | Access to this page is forbidden.'),
 Document(metadata={'source': 'https://www.artificialintelligence-news.com/news/study-finds-ai-slash-global-carbon-emissions/'}, page_content='403 - Forbidden | Access to this page is forbidden.'),
 Document(metadata={'source': 'https://www.artificialintelligence-news.com/news/nvidia-reclaims-title-of-most-valuable-company-on-ai-momentum/'}, page_content='403 - Forbidden | Access to this page is forbidden.'),
 Document(metadata={'source': 'https://www.artificialintelligence-news.com/news/meta-will-train-ai-models-using-eu-user-data/'}, page_content='403 - Forbidden | Access to this page is forbidden.')]

In [63]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [65]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len([t.page_content for t in text_chunks]))]
vector_store.add_documents(documents=text_chunks, ids=uuids)


['1369a93c-3cee-4890-ba2f-ad339812263e',
 '30b473ef-946c-45ed-a36b-801e40839701',
 'fcef2a8e-d12c-467f-b8c1-c2e6efb08c57',
 'e3aa09de-d55e-40fc-b3f6-cd0cba1bf094']

In [67]:
notebook_login()


In [68]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [69]:
tokenizer = AutoTokenizer.from_pretrained(model,use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(model,device_map='auto',torch_dtype=torch.float16,use_auth_token=True,load_in_8bit=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:711: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.

In [ ]:
pipe = pipeline("text-generation",model=model,tokenizer=tokenizer,torch_dtype=torch.bfloat16,device_map='auto',max_new_tokens=512,do_sample=True,top_k=30
                ,num_return_sequences=1,eos_token_id=tokenizer.eos_token_id)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe,model_kwargs={'temperature':0})

In [ ]:
llm.predict("provide me a summary of the harry potter")

In [ ]:
from langchain.chains import RetrievalQA
query = "most valuable company in ai"

In [ ]:
docs = vector_store.similarity_search(query,k=3)

In [ ]:
docs

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm,chain_type='stuff',retriever=vector_store.as_retriever())

In [ ]:
query = "news about robinhood"
qa.run(query)